In [ ]:
import os
os.chdir("..")
from src.dataset import VietnameseTextDataset, prepare_vietnamese_dataset, load_texts_from_folder
from tokenizers import Tokenizer
import torch
from glob import glob
import torch.nn as nn

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Punctuation, Sequence, Digits, Metaspace
from tokenizers.normalizers import NFC, NFD, Sequence as NormalizerSequence
from tokenizers.processors import TemplateProcessing
import os
import glob
from src.tokenizer import VietnameseTokenizer, VietnamesePreprocessor

In [ ]:
import torch
device = 'mps' if torch.mps.is_available() else 'cpu'
device

In [ ]:
input_filename = 'train_data_1/vietnam_poetry.txt'   # your input file
output_filename = 'train_data_1/vietnam_poetry_cleaned.txt' # desired output file
lower_filename = 'train_data_1/vietnam_poetry_lower.txt'
segmented_filename = "data1/segmented_output1.txt"

In [ ]:
with open(segmented_filename, "r", encoding="utf-8") as f:
    segmented_output = f.read()

In [ ]:
segmented_output = segmented_output.split("<s>")

In [ ]:
for i, text in enumerate(segmented_output):
    segmented_output[i] = segmented_output[i].lstrip()

In [ ]:
cnt=0
for text in segmented_output[1:]:
    if text.endswith("[LF]"):
        cnt+=1
cnt

In [ ]:
with open("data1/segmented_output2.txt", "w", encoding="utf-8") as f:
    for text in segmented_output:
        f.write("<s>" + text + "\n")

In [ ]:
preprocessor = VietnamesePreprocessor()

In [ ]:
vn_tokenizer = VietnameseTokenizer()
print(segmented_output[1])
preprocessor.clean_text(segmented_output[1])

In [ ]:
raw_texts = load_texts_from_folder("data1")

In [ ]:
all_sentences = []
for text in raw_texts:
    cleaned_text = preprocessor.clean_text(text)
    sentences_from_file = preprocessor.segment_sentences(cleaned_text)
    all_sentences.extend(
        sentences_from_file
    )  # Use extend to add all sentences to one list


In [ ]:
all_sentences[1]

In [5]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

segment_tokenizer = AutoTokenizer.from_pretrained("NlpHUST/vi-word-segmentation", model_max_length=512, truncation=True)
model_segment = AutoModelForTokenClassification.from_pretrained("NlpHUST/vi-word-segmentation")

# segment_tokenizer.model_max_length = 512

nlp = pipeline("token-classification", model=model_segment, tokenizer=segment_tokenizer, device=device)

async def word_segment(text: str) -> str:
    ner_results = nlp(text)
    example_tok = ""
    for e in ner_results:
        if "##" in e["word"]:
            example_tok = example_tok + e["word"].replace("##","")
        elif e["entity"] =="I":
            example_tok = example_tok + "_" + e["word"]
        else:
            example_tok = example_tok + " " + e["word"]
    return example_tok

Device set to use mps


In [ ]:
with open(input_filename, "r", encoding="utf-8") as f:
    data = f.read()
data = data.lower()

In [ ]:
with open(lower_filename, "w", encoding="utf-8") as f:
    f.write(data)

In [ ]:
with open(lower_filename, "r", encoding="utf-8") as f:
    data = f.readlines()
len(data)

In [ ]:
import unicodedata

async def is_lowercase_letter(char: str):
    return unicodedata.category(char) == "Ll"

async def capitalize(text: str):
    if not text:
        return text
    
    first_char = text[0]
    
    if await is_lowercase_letter(first_char):
        # Chuẩn hóa để đảm bảo nhất quán (NFC)
        normalized = unicodedata.normalize('NFC', text)
        return normalized.capitalize()
    else:
        return text

import re

async def clean_text(text):
    # Giữ lại: chữ cái, số, dấu câu cơ bản, khoảng trắng
    # Loại bỏ: emoji, icon, ký tự đặc biệt
    cleaned = re.sub(r'[^\w\s\.\,\!\?\;\:\-\n]', '', text)
    return cleaned

In [ ]:
for i, text in enumerate(data):
    if text.startswith("<s>thơ"):
        # text = text.strip()
        continue
    text = text.lstrip()
    text = await capitalize(text)
    text = await clean_text(text)
    data[i] = text

In [ ]:
with open(output_filename, "w", encoding="utf-8") as f:
    for text in data:
        f.write(text)

In [ ]:
with open(output_filename, "r", encoding="utf-8") as f:
    clean_data = f.read()
    data = f.readlines()

print(len(clean_data), len(data))

In [ ]:
clean_data = clean_data.split("<s>")
len(clean_data)

In [ ]:
clean_data[1]

In [ ]:
from datasets import Dataset, Sequence, Value
dataset = Dataset.from_list([{"text": text} for text in clean_data])

In [ ]:
dataset

In [ ]:
def process_batch(batch):
    # Gọi pipeline với batch text
    results = nlp(batch["text"])  # pipeline tự xử lý batch

    segmented_texts = []
    for result in results:
        example_tok = ""
        for e in result:
            if "##" in e["word"]:
                example_tok = example_tok + e["word"].replace("##","")
            elif e["entity"] =="I":
                example_tok = example_tok + "_" + e["word"]
            else:
                example_tok = example_tok + " " + e["word"]
        segmented_texts.append(example_tok)
    
    return {"segmented_text": segmented_texts}

In [ ]:
dataset_with_segmented = dataset.map(process_batch, batched=True, batch_size=256)

In [ ]:
dataset_with_segmented[2]

In [ ]:
segmented_outputs = dataset_with_segmented["segmented_text"]

In [ ]:
segmented_outputs[3]

In [ ]:
import re

def add_newline_before_capital(text):
    # Dùng regex: tìm các từ bắt đầu bằng chữ cái in hoa (trừ vị trí đầu chuỗi)
    # \b = word boundary, [A-ZÀÁẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬĐEÉÈẺẼẸÊẾỀỂỄỆIÍÌỈĨỊOÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢUÚÙỦŨỤƯỨỪỬỮỰYÝỲỶỸỴ]
    # Nhưng đơn giản: [A-Z] + các chữ cái tiếng Việt in hoa

    # Danh sách chữ cái tiếng Việt in hoa
    uppercase_vietnamese = 'AÁÀẠẢÃĂẮẰẶẲẴÂẤẦẬẨẪBCDĐEÉÈẸẺẼÊẾỀỆỂỄFGHIÍÌỊỈĨKLMNOÓÒỌỎÕÔỐỒỘỔỖƠỚỜỢỞỠPQRSTUÚÙỤỦŨƯỨỪỰỬỮVWXYÝỲỴỶỸ'
    pattern = f'(?<!^)(\\b[{uppercase_vietnamese}][a-zàáảãạăắằẳẵặâấầẩẫậđèéẻẽẹêếềểễệíìỉĩịóòỏõọôốồổỗộơớờởỡợùúủũụưứừửữựýỳỷỹỵ_]+)'

    # Thay thế: thêm \n trước từ in hoa (trừ từ đầu)
    result = re.sub(pattern, r'\n\1', text)
    return result

In [ ]:
segmented_output = []
for text in segmented_outputs:
    new_text = add_newline_before_capital(text)
    segmented_output.append(new_text)

In [ ]:
segmented_output[1]

In [ ]:
output_file = "train_data_1/segmented_output.txt"
with open(output_file, 'w', encoding='utf-8') as f:
    for line in segmented_output:
        f.write("<s>" + line + "\n")

print(f"✅ Đã xử lý và lưu {len(segmented_output)} dòng vào '{output_file}'")

In [ ]:
async def preprocessing():
    with open(output_filename, "w", encoding="utf-8") as outfile:
        for text in data:
            if text.startswith("<s>thơ"):
                outfile.write(text)
                continue
            text = text.strip()
            # text = unicodedata.normalize('NFC', text)
            text = await capitalize(text)
            text = await clean_text(text)
            # text = await word_segment(text)
            outfile.write(text + "\n")

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece, BPE
from tokenizers.trainers import WordPieceTrainer, BpeTrainer
from tokenizers.pre_tokenizers import Whitespace, Sequence
from tokenizers.normalizers import Lowercase, NFD, NFC, Strip, StripAccents, Sequence as NormalizerSequence
import os
import glob

# 1. Khởi tạo tokenizer với mô hình WordPiece
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

# 2. Chuẩn hóa văn bản (rất quan trọng với tiếng Việt)
tokenizer.normalizer = NormalizerSequence([
    Strip(),  # Remove leading/trailing whitespace
    NFC(),  # Normalize Vietnamese diacritics properly
])

# ⚠️ Nếu bạn muốn giữ dấu thanh (phù hợp với thơ), hãy BỎ Lowercase và StripAccents
# → Chỉ dùng NFD() để chuẩn hóa Unicode

# 3. Pre-tokenizer: tách theo khoảng trắng và dấu câu
tokenizer.pre_tokenizer = Sequence([Whitespace()])

# 4. Trainer
trainer = BpeTrainer(
    vocab_size=25000,
    min_frequency=2,
    special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "[BOS]", "[EOS]", "[LF]"],
    show_progress=True,
    initial_alphabet = [
        # Chữ thường có dấu
        "a", "á", "à", "ả", "ã", "ạ",
        "ă", "ắ", "ằ", "ẳ", "ẵ", "ặ",
        "â", "ấ", "ầ", "ẩ", "ẫ", "ậ",
        "b", "c", "d", "đ",
        "e", "é", "è", "ẻ", "ẽ", "ẹ",
        "ê", "ế", "ề", "ể", "ễ", "ệ",
        "f", "g", "h", "i", "í", "ì", "ỉ", "ĩ", "ị",
        "k", "l", "m", "n",
        "o", "ó", "ò", "ỏ", "õ", "ọ",
        "ô", "ố", "ồ", "ổ", "ỗ", "ộ",
        "ơ", "ớ", "ờ", "ở", "ỡ", "ợ",
        "p", "q", "r", "s", "t",
        "u", "ú", "ù", "ủ", "ũ", "ụ",
        "ư", "ứ", "ừ", "ử", "ữ", "ự",
        "v", "x", "y", "ý", "ỳ", "ỷ", "ỹ", "ỵ",

        # Chữ HOA có dấu
        "A", "Á", "À", "Ả", "Ã", "Ạ",
        "Ă", "Ắ", "Ằ", "Ẳ", "Ẵ", "Ặ",
        "Â", "Ấ", "Ầ", "Ẩ", "Ẫ", "Ậ",
        "B", "C", "D", "Đ",
        "E", "É", "È", "Ẻ", "Ẽ", "Ẹ",
        "Ê", "Ế", "Ề", "Ể", "Ễ", "Ệ",
        "F", "G", "H", "I", "Í", "Ì", "Ỉ", "Ĩ", "Ị",
        "K", "L", "M", "N",
        "O", "Ó", "Ò", "Ỏ", "Õ", "Ọ",
        "Ô", "Ố", "Ồ", "Ổ", "Ỗ", "Ộ",
        "Ơ", "Ớ", "Ờ", "Ở", "Ỡ", "Ợ",
        "P", "Q", "R", "S", "T",
        "U", "Ú", "Ù", "Ủ", "Ũ", "Ụ",
        "Ư", "Ứ", "Ừ", "Ử", "Ữ", "Ự",
        "V", "X", "Y", "Ý", "Ỳ", "Ỷ", "Ỹ", "Ỵ"
    ],
    continuing_subword_prefix="##"  # Dùng cho subword tiếp theo (ví dụ: "sinh" → "sin", "##h")
)

# 5. Dữ liệu huấn luyện
train_files = glob.glob("./data1/*.txt")
if not train_files:
    raise FileNotFoundError("Không tìm thấy file dữ liệu trong thư mục ./train_data")

# 6. Huấn luyện
tokenizer.train(files=train_files, trainer=trainer)

# 7. Lưu tokenizer
tokenizer.save("bpe_tokenizer.json")
print("✅ WordPiece tokenizer đã được lưu vào 'wordpiece_tokenizer.json'")

In [ ]:
tokens = tokenizer.encode(await word_segment("ai ơi xa bến quê hương nhớ về quê mẹ nắng sương đây nè"), add_special_tokens=True)
tokens.ids

In [ ]:
tokenizer.decode(tokens.ids, skip_special_tokens=False)

In [ ]:
preprocessor = VietnamesePreprocessor()

In [ ]:
data_folder = "data1"
raw_texts = load_texts_from_folder(data_folder)

In [ ]:
all_sentences = []
for text in raw_texts:
    cleaned_text = preprocessor.clean_text(text)
    sentences_from_file = preprocessor.segment_sentences(cleaned_text)
    all_sentences.extend(
        sentences_from_file
    )

In [ ]:
all_sentences = []
for text in raw_texts:
    cleaned_text = preprocessor.clean_text(text)
    sentences_from_file = preprocessor.segment_sentences(cleaned_text)
    all_sentences.extend(
        sentences_from_file
    )  # Use extend to add all sentences to one list

# print(all_sentences)
print(f"   - Number of sentences: {len(all_sentences)}")

In [ ]:
vn_tokenizer = VietnameseTokenizer()
tokenizer = vn_tokenizer.load("wordpiece_tokenizer.json")

In [ ]:
train_dataset = VietnameseTextDataset(all_sentences, tokenizer, 512)

In [ ]:
sample = train_dataset.__getitem__(1)
ids = sample['input_ids']
print(tokenizer.decode(ids.tolist(), skip_special_tokens=False))

In [ ]:
# Build tokenizer with Vietnamese-specific settings
trainer = vn_tokenizer.build_tokenizer(
    vocab_size=20000,
    min_frequency=2
)

# Get training files
train_files = glob.glob(os.path.join("./data1", "*.txt"))

if not train_files:
    print("No training files found in ./train_data/")
    print("Please add Vietnamese text files (.txt) to ./train_data/ directory")
    exit()

print(f"Found {len(train_files)} training files")

# Train the tokenizer
print("Training tokenizer...")
vn_tokenizer.train(train_files, trainer)

# Setup post-processor
vn_tokenizer.setup_post_processor()

# Save tokenizer
vn_tokenizer.save("vietnamese_enhanced_tokenizer.json")
print("Tokenizer saved as 'vietnamese_enhanced_tokenizer.json'")

# Test the tokenizer
vn_tokenizer.test_tokenizer()

In [ ]:
def test_tokenize(tokenizer, text: str):
    text = word_segment(text)
    tokens = tokenizer.encode(text).ids
    return tokens

In [ ]:
tokens = test_tokenize(vn_tokenizer, "ai ơi xa bến quê hương nhớ về quê mẹ nắng sương đây nè")

In [ ]:
vn_tokenizer.decode(tokens)